In [ ]:
import tensorflow as tf

import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

#Download the dataset
URL='https://github.com/Horea94/Fruit-Images-Dataset/archive/master.zip'

path_to_zip = tf.keras.utils.get_file('master.zip', origin=URL, extract=True)

In [ ]:
PATH = os.path.join(os.path.dirname(path_to_zip), 'Fruit-Images-Dataset-master')

train_dir = os.path.join(PATH, 'Training')
validation_dir = os.path.join(PATH, 'Test')

In [ ]:
#Select the classes we are interested in.
dir = pathlib.Path(validation_dir)

CLASS_NAMES = np.array([item.name for item in dir.glob('*')])

reg=re.compile('^(Apple|Banana|Plum|Pepper|Cherry|Grape|Tomato|Potato|Pear|Peach).*')

temporary_class_names=[]

for c in CLASS_NAMES:
  if reg.match(c):
    temporary_class_names.append(c)

CLASS_NAMES_MOST_USED=temporary_class_names

In [ ]:
BATCH_SIZE = 50
IMG_HEIGHT = 32
IMG_WIDTH = 32

In [ ]:
#create a dataset containing the training set we are intrested in.
list_ds=[]

for c in CLASS_NAMES_MOST_USED:
  train_dirs = os.path.join(train_dir, c)
  train_dirs = pathlib.Path(train_dirs)
  list_ds.append(tf.data.Dataset.list_files(str(train_dirs/'*')))

train_ds=list_ds[0].concatenate(list_ds[1])
for i in range(2,len(list_ds)):
  train_ds=train_ds.concatenate(list_ds[i])

#Count the number of example in the training set.
num=0
for j in train_ds.as_numpy_iterator():
  num=num+1
TRAINING_NUMBER=num


#create a dataset containing the test set we are intrested in.
list_ds=[]

for c in CLASS_NAMES_MOST_USED:
  test_dirs = os.path.join(validation_dir, c)
  test_dirs = pathlib.Path(test_dirs)
  list_ds.append(tf.data.Dataset.list_files(str(test_dirs/'*')))

test_ds=list_ds[0].concatenate(list_ds[1])
for i in range(2,len(list_ds)):
  test_ds=test_ds.concatenate(list_ds[i])

#Count the number of example in the test set.
num=0
for j in test_ds.as_numpy_iterator():
  num=num+1
TEST_NUMBER=num

In [ ]:
#return the label associated to the image.
def decode_label(parts):

  if tf.strings.regex_full_match(parts, '.*Apple.*'):
    return tf.constant(0)
  if tf.strings.regex_full_match(parts, '.*Banana.*'):
    return tf.constant(1)
  if tf.strings.regex_full_match(parts, '.*Plum.*'):
    return tf.constant(2)
  if tf.strings.regex_full_match(parts, '.*Pepper.*'):
    return tf.constant(3)
  if tf.strings.regex_full_match(parts, '.*Cherry.*'):
    return tf.constant(4)
  if tf.strings.regex_full_match(parts, '.*Grape.*'):
    return tf.constant(5)
  if tf.strings.regex_full_match(parts, '.*Tomato.*'):
    return tf.constant(6)
  if tf.strings.regex_full_match(parts, '.*Potato.*'):
    return tf.constant(7)
  if tf.strings.regex_full_match(parts, '.*Pear.*'):
    return tf.constant(8)
  if tf.strings.regex_full_match(parts, '.*Peach.*'):
    return tf.constant(9)
  
  return tf.constant(-1)

#return the image.
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

# return the image and the associated label given the file path.
def process_path(file_path):
  label = decode_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

In [ ]:
training_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
testing_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

# Shuffle the images in the training set, and divide the images in the two sets in batch.
ds_for_training=training_ds.shuffle(TRAINING_NUMBER).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds_for_test=testing_ds.batch(TEST_NUMBER).prefetch(buffer_size=AUTOTUNE)

In [ ]:
#Compute the error of a predictor on a dataset using the zero one loss function.
def zero_one_loss(model,ds):
  probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

  images,labels=next(iter(ds))
  predictions = probability_model.predict(images)

  zero_one_loss=0
  for i in range(0,len(predictions)):
    if labels[i]!=np.argmax(predictions[i]):
      zero_one_loss=zero_one_loss+1

  res=zero_one_loss/(len(predictions))
  return res

In [ ]:
acc1=[]

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(32, 32 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

for i in range(5):
  model.fit(x=ds_for_training,
            epochs=1)
  
  test_loss=zero_one_loss(model,ds_for_test)

  acc1.append(test_loss)
  print('epoch: ',i+1)
  print(test_loss)

In [ ]:
acc2=[]

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(32, 32 ,3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

for i in range(5):
  model.fit(x=ds_for_training,
            epochs=1)
  
  test_loss=zero_one_loss(model,ds_for_test)

  acc2.append(test_loss)
  print('epoch: ',i+1)
  print(test_loss)

In [ ]:
acc3=[]

BATCH_NUMBER=int(round(TRAINING_NUMBER/50))
VALIDATION_NUMBER=int(BATCH_NUMBER/4)
hyper_ds_train=ds_for_training.take(BATCH_NUMBER-VALIDATION_NUMBER)
hyper_ds_validation=ds_for_training.skip(BATCH_NUMBER-VALIDATION_NUMBER)
hyper_ds_validation=hyper_ds_validation.unbatch().batch(VALIDATION_NUMBER)

best_loss=1


for num_layer_dense in range(1,3):
  for num_node_layer in range(1,3):
    for drop_value in range(2):
      model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(32, 32 ,3)),
        tf.keras.layers.MaxPooling2D()])
    
      model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D())
      model.add(tf.keras.layers.Dropout(drop_value/10))
      model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D())
      model.add(tf.keras.layers.Dropout(drop_value/10))
      model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D())
      model.add(tf.keras.layers.Dropout(drop_value/10))
      model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
      model.add(tf.keras.layers.MaxPooling2D())
      model.add(tf.keras.layers.Dropout(drop_value/10))
    
      model.add(tf.keras.layers.Flatten())

      for i in range(num_layer_dense):
        model.add(tf.keras.layers.Dense(256*num_node_layer, activation='relu'))
        model.add(tf.keras.layers.Dense(256*num_node_layer, activation='relu'))

      tf.keras.layers.Dense(10)

      model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

      model.fit(x=hyper_ds_train,
                epochs=2)

      temp_loss=zero_one_loss(model,hyper_ds_validation)
      if temp_loss<=best_loss:
        best_loss=temp_loss
        best_model=model

for i in range(5):
  best_model.fit(x=ds_for_training,
            epochs=1)
  
  test_loss=zero_one_loss(best_model,ds_for_test)

  acc3.append(test_loss)
  print('epoch: ',i+1)
  print(test_loss)

In [ ]:
#plot the results
epochs_range = range(1,6)

plt.figure(figsize=(4, 2))
plt.plot(epochs_range, acc1, label='Test Error first network')
plt.plot(epochs_range, acc2, label='Test Error second network')
plt.plot(epochs_range, acc3, label='Test Error third network')
plt.legend(loc='upper right', prop={'size': 8})
plt.ylabel('Test error')
plt.xlabel('Epoch')
plt.title('Test error')
plt.show()